<a href="https://colab.research.google.com/github/Datangels/COVID19_analysis/blob/master/CoronaVirusITALY_GLOBAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DEFINE LIBRARIES**

In [0]:
import pandas as pd
import numpy as np
import io
import requests
from google.colab import files
pd.options.mode.chained_assignment = None
! pip install geopandas

# **DOWNLOAD REGION DATA**

In [0]:
url_first_part = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'

file_dates = ['02-02-2020.csv', '02-03-2020.csv', '02-04-2020.csv',
              '02-05-2020.csv', '02-06-2020.csv', '02-07-2020.csv', '02-08-2020.csv', '02-09-2020.csv', '02-10-2020.csv', '02-11-2020.csv',
              '02-12-2020.csv', '02-13-2020.csv', '02-14-2020.csv', '02-15-2020.csv', '02-16-2020.csv', '02-17-2020.csv', '02-18-2020.csv',
              '02-19-2020.csv', '02-20-2020.csv', '02-21-2020.csv', '02-22-2020.csv', '02-23-2020.csv', '02-24-2020.csv', '02-25-2020.csv',
              '02-26-2020.csv', '02-27-2020.csv', '02-28-2020.csv', '02-29-2020.csv', '03-01-2020.csv', '03-02-2020.csv', '03-03-2020.csv',
              '03-04-2020.csv', '03-05-2020.csv', '03-06-2020.csv', '03-07-2020.csv', '03-08-2020.csv', '03-09-2020.csv', '03-10-2020.csv',
              '03-11-2020.csv', '03-12-2020.csv', '03-13-2020.csv', '03-14-2020.csv', '03-15-2020.csv', '03-16-2020.csv', '03-17-2020.csv',
              '03-18-2020.csv', '03-19-2020.csv', '03-20-2020.csv', '03-21-2020.csv', '03-22-2020.csv', '03-23-2020.csv']


full_data_list = []

for i in range(0, len(file_dates)):
  # print('Processing: ' + file_dates[i])
  url_complete = url_first_part + file_dates[i]
  response = requests.get(url_complete).content
  temp_df = pd.read_csv(io.StringIO(response.decode('utf-8')))
  try:
    temp_df['Last Update'] = temp_df['Last_Update']
    temp_df['Country/Region'] = temp_df['Country_Region']
    temp_df['Province/State'] = temp_df['Province_State']
  except:
    pass
  full_data_list.append(temp_df)

global_df = pd.concat(full_data_list, ignore_index=True)

# global_df.to_csv('global_df.csv')
# files.download('province_df.csv')

In [0]:
# global_df['Last Update'].unique()
global_df[global_df['Last_Update'].isnull() == True]

# **FIX AND ADAPT DATA**

In [0]:
global_df_to_manage = global_df

global_df_to_manage = global_df_to_manage.fillna(0)

global_df_to_manage['Last Update'] = global_df_to_manage['Last Update'].str[:10]

global_df_to_manage = global_df_to_manage.groupby(['Last Update', 'Country/Region']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()

global_df_to_manage['days_from_last_obs'] = (pd.to_datetime(global_df_to_manage['Last Update'].max()) - pd.to_datetime(global_df_to_manage['Last Update'], errors='coerce')).dt.days

In [0]:
list(set(global_df_to_manage[(global_df_to_manage['Last Update'] == '2020-03-18')]['days_from_last_obs']))

In [0]:
global_df_to_manage[(global_df_to_manage['days_from_last_obs'] == 9) & (global_df_to_manage['Country/Region'] == 'Italy')]

# **ANALYZE DATA**

In [0]:
kpi_columns = ['Confirmed', 'Deaths', 'Recovered']

global_df_to_manage_daily = pd.DataFrame()

i = 0
while i < 10:
  temp_global_daily_df = global_df_to_manage[(global_df_to_manage['days_from_last_obs'] == i) | (global_df_to_manage['days_from_last_obs'] == i + 1)]
  temp_global_daily_df['days_range'] = str(i) + '__' + str(i + 1)
  temp_global_daily_df.sort_values(by=['Country/Region', 'days_from_last_obs'], ascending=[False, False], inplace=True)  
  for q in range(0, len(kpi_columns)):
    temp_global_daily_df['d_' + kpi_columns[q]] = temp_global_daily_df.groupby(['Country/Region'])[kpi_columns[q]].diff().fillna(0).replace([np.inf, -np.inf], 0)
    temp_global_daily_df['dp_' + kpi_columns[q]] = temp_global_daily_df.groupby(['Country/Region'])[kpi_columns[q]].pct_change().apply(lambda x: round(x, 2)* 100).fillna(0).replace([np.inf, -np.inf], 0)
  global_df_to_manage_daily = global_df_to_manage_daily.append(temp_global_daily_df)
  i = i + 1

In [0]:
global_df_to_manage_daily[(global_df_to_manage_daily['days_from_last_obs'] == 5) & (global_df_to_manage_daily['Country/Region'] == 'Italy')]

# **SHOW RECAP**

In [0]:
COLUMNS_TO_DISPLAY = ['days_from_last_obs', 'Last Update', 'Country/Region', 'Confirmed', 'd_Confirmed', 'dp_Confirmed']
DAYS_AGO = 1 # 0 is the last data point

global_df_to_manage_daily_recap = global_df_to_manage_daily[COLUMNS_TO_DISPLAY]
# global_df_to_manage_daily_recap[global_df_to_manage_daily_recap['days_from_last_obs'] == DAYS_AGO].sort_values('Country/Region')

global_df_to_manage_daily_recap[(global_df_to_manage_daily_recap['days_from_last_obs'] == DAYS_AGO) & (global_df_to_manage_daily_recap['Country/Region'] == 'Italy')]

## ARRIVATO QUI I DATI RIGUARDO UNITED KINGDOM NON TORNANO

# **SHOW PIVOT**

In [0]:
PIVOT_MAIN_KPI = 'dp_totale_casi'

province_df_to_manage_daily_for_pivot = province_df_to_manage_daily.groupby(['denominazione_provincia', 'days_range']).last().reset_index()
province_df_to_manage_daily_pivot = province_df_to_manage_daily_for_pivot.pivot(index='denominazione_provincia', columns='days_range', values=[PIVOT_MAIN_KPI]).fillna(0)
province_df_to_manage_daily_pivot[province_df_to_manage_daily_pivot.columns[::-1]]

# **PLOT GEODATA**

In [0]:
import os
import numpy as np
import folium
from folium import plugins
import geopandas as gpd

In [0]:
# Download Geodata
file_name = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson'
province_geo_data = gpd.read_file(file_name)
province_geo_data['prov_name'] = province_geo_data['prov_name'].str.lower()

In [0]:
KPI_MAP = 'dp_totale_casi'
DAYS_AGO = 0 # 0 is the last data point

map_italy = folium.Map([41.9028, 12.4964], zoom_start=6)

popups = province_df_to_manage_daily[province_df_to_manage_daily['days_from_last_obs'] == DAYS_AGO]
popups = popups[['denominazione_provincia', KPI_MAP]]

popups.columns = ['prov_name', KPI_MAP]
popups['prov_name'] = popups['prov_name'].str.lower()

province_geo_data_short = province_geo_data.merge(popups,on="prov_name")

province_geo_data_short = province_geo_data_short[['prov_name', KPI_MAP, 'geometry']]

folium.Choropleth(
    geo_data=province_geo_data_short,
    data=province_geo_data_short,
    columns=['prov_name', KPI_MAP],
    key_on='feature.properties.prov_name',
    fill_color='YlOrRd'
).add_to(map_italy)

import branca.colormap as cm
colormap = cm.linear.YlGnBu_09.to_step(data=popups[KPI_MAP], method='quant', quantiles=[0,0.1,0.75,0.9,0.98,1])

style_function = lambda x: {
    'fillColor': '#ffffff',
    'color':'#000000', 
    'fillOpacity': 0.1,
    'weight': 0.1}
highlight_function = lambda x: {
    'fillColor': '#000000',
    'color':'#000000',
    'fillOpacity': 0.50,
    'weight': 0.1}
NIL = folium.features.GeoJson(
    province_geo_data_short,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['prov_name', KPI_MAP],
        aliases=['prov_name: ', KPI_MAP + ' : '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
map_italy.add_child(NIL)
map_italy.keep_in_front(NIL)

map_italy